# Gradio and Chatbots

In [ ]:
import gradio as gr


# Generic Chatbot

In [ ]:
gr.load_chat("http://localhost:11434/v1/", model="gemma3:1b", token="***").launch()

# Apple Support

In [ ]:
# app.py
import os
import gradio as gr
from openai import OpenAI

SYSTEM_PROMPT = (
    "You are a polite, concise virtual assistant for Apple Support. "
    "Offer troubleshooting steps for Apple devices (iPhone, iPad, Mac, Watch, AirPods). "
    "Do not request personal data or serial numbers. "
    "For account, billing, or repair needs, direct users to the official Apple Support channels."
)

BASE_URL = "http://localhost:11434/v1/"
MODEL = "gemma3:1b"
API_KEY = os.getenv("OLLAMA_TOKEN", "ollama")

client = OpenAI(base_url=BASE_URL, api_key=API_KEY)

def normalize_history(history):
    """
    Supports multiple Gradio history formats:
    1) [(user, assistant), ...]
    2) [{"role":"user","content":"..."}, {"role":"assistant","content":"..."}, ...]
    3) [{"role":"user","content":"..."}, ...] (some variants)
    4) [{"type":"human"/"ai","content":"..."}] (older/internal variants)
    """
    messages = []

    if not history:
        return messages

    # Case: messages already
    if isinstance(history, list) and isinstance(history[0], dict):
        for m in history:
            role = m.get("role")
            content = m.get("content")

            # Some variants use "type" instead of "role"
            if role is None and "type" in m:
                role = "user" if m["type"] in ("human", "user") else "assistant"

            if role in ("user", "assistant") and content is not None:
                messages.append({"role": role, "content": content})
        return messages

    # Case: tuple pairs
    if isinstance(history, list):
        for item in history:
            # could be (user, assistant) or sometimes longer tuples
            if isinstance(item, (tuple, list)) and len(item) >= 2:
                user_msg = item[0]
                assistant_msg = item[1]
                if user_msg:
                    messages.append({"role": "user", "content": str(user_msg)})
                if assistant_msg:
                    messages.append({"role": "assistant", "content": str(assistant_msg)})
        return messages

    return messages

def chat_fn(message, history):
    messages = [{"role": "system", "content": SYSTEM_PROMPT}]
    messages.extend(normalize_history(history))
    messages.append({"role": "user", "content": message})

    resp = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        temperature=0.3,
    )
    return resp.choices[0].message.content

demo = gr.ChatInterface(
    fn=chat_fn,
    title="Apple Support Assistant",
    description="Polite, concise troubleshooting help for Apple devices.",
)

if __name__ == "__main__":
    demo.launch()


# Financial Advisor and Investment advisor chatbot

In [ ]:
# app.py
import os
import gradio as gr
from openai import OpenAI

SYSTEM_PROMPT = (
    "You are a knowledgeable, calm, and structured virtual advisor for users in the United States, "
    "specializing in personal finance, retirement planning, life insurance, annuities, "
    "tax-aware investing, and stock market portfolio construction. "
    "\n\n"
    "PRIMARY OBJECTIVE:\n"
    "1) Understand the user’s goal(s) and constraints.\n"
    "2) Derive a risk profile (risk tolerance + risk capacity) and time horizon.\n"
    "3) Provide specific, actionable options such as model allocations, example tickers/ETFs, "
    "and insurance/annuity product-type recommendations appropriate to the stated profile.\n"
    "\n\n"
    "DISCOVERY (ASK BEFORE RECOMMENDING):\n"
    "Ask concise questions to determine:\n"
    "- Goal: retirement, income, growth, preservation, education, home, etc.\n"
    "- Time horizon(s): <3y, 3–7y, 7–15y, 15y+.\n"
    "- Risk tolerance: comfort with drawdowns (e.g., -10%, -20%, -35%), and reaction to volatility.\n"
    "- Risk capacity: income stability, emergency fund, debt, near-term cash needs.\n"
    "- Current situation: age range, employment, dependents, existing accounts (401k/IRA/HSA/taxable), "
    "and approximate investable amount.\n"
    "- Tax context (USA): filing status, bracket range, state tax presence, capital gains horizon.\n"
    "- Preferences/constraints: ESG, dividends, no-crypto, avoid single stocks, etc.\n"
    "\n\n"
    "RISK PROFILING OUTPUT:\n"
    "Summarize the derived profile as one of: Conservative, Moderately Conservative, Balanced, "
    "Growth, Aggressive Growth, and explain why (time horizon + drawdown tolerance + capacity). "
    "\n\n"
    "PORTFOLIO GUIDANCE (BE SPECIFIC):\n"
    "Provide 2–3 specific model portfolio options aligned to the risk profile using diversified, "
    "liquid instruments typically available to U.S. investors (e.g., broad-market ETFs, bond ETFs, "
    "T-bill ETFs/money market guidance). Include example ticker symbols and target percentages, "
    "plus simple rebalancing rules (e.g., quarterly/annual or threshold-based). "
    "Discuss tradeoffs (cost, diversification, volatility, sequence-of-returns risk). "
    "Avoid leverage unless the user explicitly asks and demonstrates sophistication.\n"
    "\n\n"
    "INSURANCE & ANNUITIES GUIDANCE (BE SPECIFIC ON TYPES):\n"
    "If the user’s goal includes protection or guaranteed income, recommend suitable product TYPES "
    "(e.g., term life vs permanent, SPIA/DIA/fixed annuity/fixed indexed annuity/variable annuity) "
    "and explain when each is appropriate, key riders (income rider, LTC rider), key fees, surrender "
    "periods, and the primary risks. Do not invent insurer-specific quotes. If costs/returns are needed, "
    "use ranges and clearly label as estimates.\n"
    "\n\n"
    "USA TAX-AWARE GUIDANCE:\n"
    "Provide high-level tax planning ideas relevant to the user’s profile: "
    "401(k)/403(b)/457, Traditional vs Roth IRA, Backdoor Roth (when relevant), HSA, "
    "tax-loss harvesting, asset location (bonds in tax-advantaged, equities in taxable), "
    "qualified dividends, long-term capital gains timing, and estimated tax considerations "
    "for self-employment. "
    "For LLC topics, explain when an LLC may help (liability, clean bookkeeping) and when tax benefits "
    "come from the tax election/structure (sole prop vs S-corp election), payroll requirements, "
    "reasonable compensation concept, and compliance costs. Always advise confirming with a CPA/EA.\n"
    "\n\n"
    "COMPLIANCE & SAFETY RULES:\n"
    "- You must include a brief disclaimer that you provide educational information and model examples, "
    "not individualized legal/tax advice.\n"
    "- You must not guarantee returns or claim certainty.\n"
    "- You must not request highly sensitive personal data (SSN, account numbers).\n"
    "- If the user has complex situations (large net worth, tax residency issues, trusts, estate planning, "
    "business sale, margin/leverage), recommend consulting a licensed professional.\n"
    "\n\n"
    "COMMUNICATION STYLE:\n"
    "Be concise, structured, and practical. Use bullets and small tables when helpful. "
    "When recommending, provide: (a) why it fits, (b) the specific allocation/options, "
    "(c) key risks, (d) next steps the user can take today."
)

BASE_URL = "http://localhost:11434/v1/"
MODEL = "gemma3:1b"
API_KEY = os.getenv("OLLAMA_TOKEN", "ollama")

client = OpenAI(base_url=BASE_URL, api_key=API_KEY)

def normalize_history(history):
    """
    Supports multiple Gradio history formats:
    1) [(user, assistant), ...]
    2) [{"role":"user","content":"..."}, {"role":"assistant","content":"..."}, ...]
    3) [{"role":"user","content":"..."}, ...] (some variants)
    4) [{"type":"human"/"ai","content":"..."}] (older/internal variants)
    """
    messages = []

    if not history:
        return messages

    # Case: messages already
    if isinstance(history, list) and isinstance(history[0], dict):
        for m in history:
            role = m.get("role")
            content = m.get("content")

            # Some variants use "type" instead of "role"
            if role is None and "type" in m:
                role = "user" if m["type"] in ("human", "user") else "assistant"

            if role in ("user", "assistant") and content is not None:
                messages.append({"role": role, "content": content})
        return messages

    # Case: tuple pairs
    if isinstance(history, list):
        for item in history:
            # could be (user, assistant) or sometimes longer tuples
            if isinstance(item, (tuple, list)) and len(item) >= 2:
                user_msg = item[0]
                assistant_msg = item[1]
                if user_msg:
                    messages.append({"role": "user", "content": str(user_msg)})
                if assistant_msg:
                    messages.append({"role": "assistant", "content": str(assistant_msg)})
        return messages

    return messages

def chat_fn(message, history):
    messages = [{"role": "system", "content": SYSTEM_PROMPT}]
    messages.extend(normalize_history(history))
    messages.append({"role": "user", "content": message})

    resp = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        temperature=0.3,
    )
    return resp.choices[0].message.content

demo = gr.ChatInterface(
    fn=chat_fn,
    title="Financial Advisor and Investment Assistant",
    description="Polite, concise financial advice and investment assistance.",
)

if __name__ == "__main__":
    demo.launch()
